# WHAT IS THIS?
This is the lmul.ipynb notebook uploaded in #things channel, but just pushed onto the github for comparison reasons

In [19]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [20]:
from hardware_accelerators.dtypes import *
from hardware_accelerators.rtllib.lmul import *
from hardware_accelerators.rtllib.multipliers import *
from hardware_accelerators.rtllib.adders import *
from hardware_accelerators.rtllib.utils.lmul_utils import *
from hardware_accelerators.simulation.utils import render_waveform
from hardware_accelerators.simulation.repr_funcs import *
from hardware_accelerators.simulation import SystolicArraySimulator
from hardware_accelerators.nn import load_model, softmax
from pyrtl import *
import pyrtl
import numpy as np
from typing import Type
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
import torch

#

---


In [21]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
# Download MNIST test data
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels.numpy()


def get_activation():
    image, _ = next(iter(test_loader))
    image = image.detach().numpy().reshape(-1)
    return image

## PyRTL lmul fix


In [22]:
def lmul_fix(
    float_a: WireVector, float_b: WireVector, dtype: Type[BaseFloat], fast=False
):
    e_bits, m_bits = dtype.exponent_bits(), dtype.mantissa_bits()
    em_bits = e_bits + m_bits
    sign_a = float_a[em_bits]
    sign_b = float_b[em_bits]
    exp_a = float_a[m_bits:-1]
    exp_b = float_b[m_bits:-1]
    exp_mantissa_a = float_a[:em_bits]
    exp_mantissa_b = float_b[:em_bits]

    zero_or_subnormal = WireVector(1)
    carry_msb = WireVector(2)
    fp_out = WireVector(dtype.bitwidth())

    OFFSET_MINUS_BIAS = lmul_offset_rtl(dtype)
    MAX_VALUE = pyrtl.Const(dtype.binary_max(), bitwidth=em_bits)

    if fast:
        final_sum = carrysave_adder(
            exp_mantissa_a, exp_mantissa_b, OFFSET_MINUS_BIAS, final_adder=kogge_stone
        )
    else:
        final_sum = exp_mantissa_a + exp_mantissa_b + OFFSET_MINUS_BIAS

    carry_msb <<= final_sum[em_bits:]
    zero_or_subnormal <<= ~pyrtl.or_all_bits(exp_a) | ~pyrtl.or_all_bits(exp_b)

    with conditional_assignment:
        with zero_or_subnormal:
            fp_out |= 0
        with carry_msb == 0:
            fp_out |= 0
        with carry_msb == 1:
            fp_out |= pyrtl.concat(sign_a ^ sign_b, final_sum[:em_bits])
        with pyrtl.otherwise:
            fp_out |= pyrtl.concat(sign_a ^ sign_b, MAX_VALUE)

    return fp_out

In [23]:
from typing import Any


class FastLmul:
    def __init__(self, dtype: Type[BaseFloat]):
        self.dtype = dtype
        bits = dtype.bitwidth()
        reset_working_block()
        input_a, input_b = Input(bits, "input_a"), Input(bits, "input_b")
        self.output = Output(bits, "output")
        self.output <<= lmul_fix(input_a, input_b, dtype)
        self.sim = pyrtl.CompiledSimulation()
        self.lmul_offset = {
            # BF16: 16248,
            BF16: get_combined_offset(8, 7, True),
            # Float32: 1064828928,
            Float32: get_combined_offset(8, 23, True),
        }
        self.bitmask = {
            BF16: 0b0111111111111111,
            Float32: 0b01111111111111111111111111111111,
        }

    def run(self, a: float, b: float) -> float:
        self.sim.step(
            {"input_a": self.dtype(a).binint, "input_b": self.dtype(b).binint},
        )
        return float(self.dtype(binint=self.sim.inspect("output")))

    def __call__(self, a: float, b: float) -> Any:
        bin_a, bin_b = self.dtype(a).binint, self.dtype(b).binint
        sign = (bin_a >> (self.dtype.bitwidth() - 1)) ^ (
            bin_b >> (self.dtype.bitwidth() - 1)
        )
        bin_a &= self.bitmask[self.dtype]
        bin_b &= self.bitmask[self.dtype]
        if bin_a >> self.dtype.mantissa_bits() == 0:
            return 0
        if bin_b >> self.dtype.mantissa_bits() == 0:
            return 0
        binint = (bin_a + bin_b + self.lmul_offset[self.dtype]) & self.bitmask[
            self.dtype
        ]
        binint |= sign << (self.dtype.bitwidth() - 1)
        return float(self.dtype(binint=binint))


def run_lmul(a: float, b: float, dtype: Type[BaseFloat] = BF16, fn=lmul_fix):
    bits = dtype.bitwidth()

    reset_working_block()
    input_a, input_b = Input(bits, "input_a"), Input(bits, "input_b")
    output = Output(bits, "output")
    output <<= fn(input_a, input_b, dtype)

    tracer = SimulationTrace("all")
    sim = Simulation(tracer=tracer)
    sim.step({input_a: dtype(a).binint, input_b: dtype(b).binint})

    binary_result = sim.inspect(output)
    result = dtype(binint=binary_result)

    print(f"Expected: {float(dtype(a * b))}")
    print(f"{a} + {b} = {result}")
    print(format(binary_result, f"0{bits}b"))

    # if fn == lmul_fix:
    #     carry_msb = sim.inspect("carry_msb")
    #     zero_flag = sim.inspect('zero_flag')
    #     print(f"carry_msb: {carry_msb}")
    #     print(f"zero_flag: {zero_flag}")

    return float(result)


def run_adder(
    a: float | int, b: float | int, dtype: Type[BaseFloat] = BF16, binary_mode=False
):
    bits = dtype.bitwidth()
    reset_working_block()
    input_a, input_b = Input(bits, "input_a"), Input(bits, "input_b")
    output = Output(bits, "output")
    output <<= float_adder(input_a, input_b, dtype)
    sim = Simulation()
    if binary_mode:
        sim.step({input_a: a, input_b: b})
    else:
        sim.step({input_a: dtype(a).binint, input_b: dtype(b).binint})

    binary_result = sim.inspect(output)
    result = dtype(binint=binary_result)

    print(f"Expected: {float(dtype(a + b))}")
    print(f"{float(a):.3f} + {float(b):.3f} = {float(result):.3f}")
    print(format(binary_result, f"0{bits}b"))

    return float(result)

In [24]:
run_lmul(-0.32, 0.70)

Expected: -0.2236328125
-0.32 + 0.7 = -0.216796875
1011111001011110


-0.216796875

In [25]:
3.89 * 3.38953138925e37

1.3185277104182501e+38

In [26]:
run_lmul(0, -0.063, fn=lmul_fix)

Expected: 0.0
0 + -0.063 = 0.0
0000000000000000


0.0

In [27]:
neg_0 = 0b1000000000000000
pos_0 = 0b0
adder_in = BF16(-0.0732)
adder_in

BF16(binary='1011110110010101', decimal=-0.0732, decimal_approx=-0.07275390625)

In [28]:
run_adder(neg_0, 0, binary_mode=True)

Expected: 32768.0
32768.000 + 0.000 = 0.000
0000000000000000


0.0

In [29]:
lmulcls = FastLmul(BF16)

In [30]:
%%time
lmulcls.run(0, 0.000003)

CPU times: user 213 µs, sys: 16 µs, total: 229 µs
Wall time: 222 µs


0.0

## Optimized numpy lmul


In [31]:
import numpy as np
from typing import List, Type, Tuple, Any
from hardware_accelerators.dtypes import BaseFloat, BF16, Float32


class OptimizedLmul:
    """Optimized implementation of LMUL using pure integer arithmetic"""

    def __init__(self, dtype: Type[BaseFloat]):
        self.dtype = dtype
        self.lmul_offset = {
            BF16: get_combined_offset(8, 7, True),
            Float32: get_combined_offset(8, 23, True),
        }
        self.bitmask = {
            BF16: 0b0111111111111111,
            Float32: 0b01111111111111111111111111111111,
        }
        self.bitwidth = dtype.bitwidth()
        self.mantissa_bits = dtype.mantissa_bits()

    def multiply(self, bin_a: int, bin_b: int) -> int:
        """Multiply two numbers in binint representation using LMUL algorithm"""
        # Extract sign bit
        sign = (bin_a >> (self.bitwidth - 1)) ^ (bin_b >> (self.bitwidth - 1))

        # Clear sign bits
        bin_a &= self.bitmask[self.dtype]
        bin_b &= self.bitmask[self.dtype]

        # Check for zero exponents (denormals or zero)
        if bin_a >> self.mantissa_bits == 0 or bin_b >> self.mantissa_bits == 0:
            return 0

        # Apply LMUL algorithm
        binint = (bin_a + bin_b + self.lmul_offset[self.dtype]) & self.bitmask[
            self.dtype
        ]

        # Set sign bit
        binint |= sign << (self.bitwidth - 1)

        return binint


def convert_to_binint(data: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """Convert numpy array of floats to array of binint representations"""
    # Create a vectorized function to convert each element
    vectorized_convert = np.vectorize(lambda x: dtype(x).binint)
    return vectorized_convert(data)


def convert_from_binint(data: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """Convert numpy array of binint representations back to floats"""
    # Create a vectorized function to convert each element
    vectorized_convert = np.vectorize(lambda x: float(dtype(binint=x)))
    return vectorized_convert(data)


def relu_binint(x: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """Apply ReLU to binint values by checking sign bit"""
    # For floating point in binint representation, negative numbers have the highest bit set
    sign_mask = 1 << (dtype.bitwidth() - 1)
    return np.where((x & sign_mask) == 0, x, 0)


def optimized_mlp_inference(
    inputs_batch: np.ndarray, model_weights: dict, dtype: Type[BaseFloat]
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Optimized MLP inference using integer arithmetic for batched inputs

    Args:
        inputs_batch: Batch of input vectors [batch_size, input_dim]
        model_weights: Dictionary containing model weights and biases
        dtype: The floating-point data type to use

    Returns:
        Tuple of (predicted_classes, output_probabilities)
    """
    # Initialize the optimized LMUL
    lmul = OptimizedLmul(dtype)

    # Pre-convert all weights and biases to binint representation
    fc1_weight_bin = convert_to_binint(model_weights["fc1_weight"], dtype)
    fc1_bias_bin = convert_to_binint(model_weights["fc1_bias"], dtype)
    fc2_weight_bin = convert_to_binint(model_weights["fc2_weight"], dtype)
    fc2_bias_bin = convert_to_binint(model_weights["fc2_bias"], dtype)

    # Convert inputs to binint
    inputs_bin = convert_to_binint(inputs_batch, dtype)

    batch_size = inputs_batch.shape[0]
    hidden_size = fc1_weight_bin.shape[0]
    output_size = fc2_weight_bin.shape[0]
    input_size = inputs_batch.shape[1]

    # First layer: matrix multiplication + bias + ReLU
    h1_bin = np.zeros((batch_size, hidden_size), dtype=np.int64)

    # Perform matrix multiplication with LMUL
    for b in range(batch_size):
        for i in range(hidden_size):
            acc = 0  # Accumulate in regular float for now
            for j in range(input_size):
                # Multiply using optimized LMUL
                prod = lmul.multiply(fc1_weight_bin[i, j], inputs_bin[b, j])

                # Convert product back to float for accumulation
                # In a real hardware implementation, this would be a floating-point addition
                prod_float = float(dtype(binint=prod))
                acc += prod_float

            # Convert accumulated result back to binint
            h1_bin[b, i] = dtype(acc).binint

    # Add bias (in binint space, this is still a floating-point addition)
    h1_with_bias_bin = np.zeros_like(h1_bin)
    for b in range(batch_size):
        for i in range(hidden_size):
            # Convert to float, add, then convert back to binint
            h1_float = float(dtype(binint=h1_bin[b, i]))
            bias_float = float(dtype(binint=fc1_bias_bin[i]))
            h1_with_bias_bin[b, i] = dtype(h1_float + bias_float).binint

    # Apply ReLU in binint space
    a1_bin = relu_binint(h1_with_bias_bin, dtype)

    # Second layer: matrix multiplication + bias
    h_out_bin = np.zeros((batch_size, output_size), dtype=np.int64)

    # Perform matrix multiplication with LMUL
    for b in range(batch_size):
        for i in range(output_size):
            acc = 0  # Accumulate in regular float for now
            for j in range(hidden_size):
                # Multiply using optimized LMUL
                prod = lmul.multiply(fc2_weight_bin[i, j], a1_bin[b, j])

                # Convert product back to float for accumulation
                prod_float = float(dtype(binint=prod))
                acc += prod_float

            # Convert accumulated result back to binint
            h_out_bin[b, i] = dtype(acc).binint

    # Add bias
    h_out_with_bias_bin = np.zeros_like(h_out_bin)
    for b in range(batch_size):
        for i in range(output_size):
            # Convert to float, add, then convert back to binint
            h_out_float = float(dtype(binint=h_out_bin[b, i]))
            bias_float = float(dtype(binint=fc2_bias_bin[i]))
            h_out_with_bias_bin[b, i] = dtype(h_out_float + bias_float).binint

    # Convert final layer output back to floats for softmax
    h_out_float = convert_from_binint(h_out_with_bias_bin, dtype)

    # Apply softmax (in float space)
    output_probs = np.zeros_like(h_out_float)
    for b in range(batch_size):
        exp_x = np.exp(
            h_out_float[b] - np.max(h_out_float[b])
        )  # For numerical stability
        output_probs[b] = exp_x / exp_x.sum()

    # Get predicted classes
    predicted_classes = np.argmax(output_probs, axis=1)

    return predicted_classes, output_probs

In [40]:
%load_ext pyinstrument

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [33]:
# Example usage
from tqdm import tqdm


if __name__ == "__main__":
    from hardware_accelerators.dtypes import BF16
    from hardware_accelerators.nn import load_model
    import time

    # Load the model
    model = load_model("../models/mlp_mnist_bf16.pth", "cpu")

    # Extract weights and biases
    fc1_weight = model.fc1.weight.data.numpy()
    fc1_bias = model.fc1.bias.data.numpy()
    fc2_weight = model.fc2.weight.data.numpy()
    fc2_bias = model.fc2.bias.data.numpy()

    # Store weights in a dictionary
    model_weights = {
        "fc1_weight": fc1_weight,
        "fc1_bias": fc1_bias,
        "fc2_weight": fc2_weight,
        "fc2_bias": fc2_bias,
    }

    # Get batch of input data (assuming get_batch_activations() returns multiple samples)
    batch_size = 10
    inputs_batch, labels = get_batch(batch_size)

    # Run standard NumPy inference for comparison
    start_time = time.time()

    numpy_results = []
    for i in tqdm(range(batch_size)):
        inputs = inputs_batch[i]
        h1_numpy = inputs @ fc1_weight.T + fc1_bias
        a1_numpy = np.maximum(0, h1_numpy)
        h_out_numpy = a1_numpy @ fc2_weight.T + fc2_bias
        a_out_numpy = softmax(h_out_numpy)
        predicted_class_numpy = np.argmax(a_out_numpy)
        numpy_results.append((predicted_class_numpy, a_out_numpy))

    numpy_time = time.time() - start_time
    numpy_predictions = np.array([r[0] for r in numpy_results])

    print(
        f"Standard NumPy inference time for batch of {batch_size}: {numpy_time:.4f} seconds"
    )

    # Run optimized inference using integer arithmetic
    start_time = time.time()
    predicted_classes, output_probs = optimized_mlp_inference(
        inputs_batch, model_weights, BF16
    )
    optimized_time = time.time() - start_time

    print(
        f"Optimized inference time for batch of {batch_size}: {optimized_time:.4f} seconds"
    )
    print(f"Speedup: {numpy_time / optimized_time:.2f}x")

    # Compare results
    match_count = np.sum(predicted_classes == numpy_predictions)
    print(
        f"Prediction match rate: {match_count}/{batch_size} ({match_count/batch_size*100:.2f}%)"
    )

    # Show a few examples
    print("\nSample comparisons:")
    for i in range(min(5, batch_size)):
        print(
            f"Sample {i}: NumPy predicted {numpy_predictions[i]}, Optimized predicted {predicted_classes[i]}"
        )

100%|██████████| 10/10 [00:00<00:00, 11158.03it/s]


Standard NumPy inference time for batch of 10: 0.5987 seconds
Optimized inference time for batch of 10: 7.7360 seconds
Speedup: 0.08x
Prediction match rate: 10/10 (100.00%)

Sample comparisons:
Sample 0: NumPy predicted 7, Optimized predicted 7
Sample 1: NumPy predicted 2, Optimized predicted 2
Sample 2: NumPy predicted 1, Optimized predicted 1
Sample 3: NumPy predicted 0, Optimized predicted 0
Sample 4: NumPy predicted 4, Optimized predicted 4


In [34]:
import numpy as np
import struct
from typing import Type, Tuple, Dict, Any
from hardware_accelerators.dtypes import BaseFloat, BF16, Float32


def float_to_binint_batch(values: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """
    Vectorized conversion of float values to binint representation

    Args:
        values: NumPy array of float values
        dtype: Target floating-point type (BF16 or Float32)

    Returns:
        NumPy array of binint representations
    """
    if dtype == BF16:
        # For BF16, we need to convert through float32 first
        # Get binary representation of float32 values
        float32_bits = np.frombuffer(
            struct.pack("!%df" % len(values.flatten()), *values.flatten()),
            dtype=np.uint32,
        ).reshape(values.shape)

        # Extract parts from float32 and construct BF16
        sign = (float32_bits >> 31) & 0x1
        exp = (float32_bits >> 23) & 0xFF
        mantissa = (float32_bits >> 16) & 0x7F  # Keep only top 7 bits of mantissa

        # Combine into BF16 binint (16 bits)
        return ((sign << 15) | (exp << 7) | mantissa).astype(np.uint16)

    elif dtype == Float32:
        # For Float32, we can directly convert
        return np.frombuffer(
            struct.pack("!%df" % len(values.flatten()), *values.flatten()),
            dtype=np.uint32,
        ).reshape(values.shape)

    else:
        raise ValueError(f"Unsupported dtype: {dtype}")


def binint_to_float_batch(binints: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """
    Vectorized conversion of binint representations back to float values

    Args:
        binints: NumPy array of binint representations
        dtype: Source floating-point type (BF16 or Float32)

    Returns:
        NumPy array of float values
    """
    if dtype == BF16:
        # Extract parts from BF16
        sign = (binints >> 15) & 0x1
        exp = (binints >> 7) & 0xFF
        mantissa = binints & 0x7F

        # Construct float32 representation
        float32_bits = (sign << 31) | (exp << 23) | (mantissa << 16)

        # Convert to float32
        return np.frombuffer(
            struct.pack("!%dI" % len(float32_bits.flatten()), *float32_bits.flatten()),
            dtype=np.float32,
        ).reshape(binints.shape)

    elif dtype == Float32:
        # For Float32, we can directly convert
        return np.frombuffer(
            struct.pack("!%dI" % len(binints.flatten()), *binints.flatten()),
            dtype=np.float32,
        ).reshape(binints.shape)

    else:
        raise ValueError(f"Unsupported dtype: {dtype}")


def lmul_vectorized(
    a_binint: np.ndarray, b_binint: np.ndarray, dtype: Type[BaseFloat]
) -> np.ndarray:
    """
    Vectorized implementation of LMUL algorithm

    Args:
        a_binint: NumPy array of binint representations
        b_binint: NumPy array of binint representations
        dtype: Floating-point type (BF16 or Float32)

    Returns:
        NumPy array of binint representations of results
    """
    if dtype == BF16:
        # Constants for BF16
        bitmask = 0x7FFF  # 15 bits (excluding sign)
        bitwidth = 16
        mantissa_bits = 7
        lmul_offset = get_combined_offset(8, 7, True)
    elif dtype == Float32:
        # Constants for Float32
        bitmask = 0x7FFFFFFF  # 31 bits (excluding sign)
        bitwidth = 32
        mantissa_bits = 23
        lmul_offset = get_combined_offset(8, 23, True)
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")

    # Extract sign bits
    sign_a = (a_binint >> (bitwidth - 1)) & 0x1
    sign_b = (b_binint >> (bitwidth - 1)) & 0x1

    # Compute result sign
    sign_result = sign_a ^ sign_b

    # Clear sign bits
    a_unsigned = a_binint & bitmask
    b_unsigned = b_binint & bitmask

    # Create masks for zero exponents
    a_exp = a_unsigned >> mantissa_bits
    b_exp = b_unsigned >> mantissa_bits

    # Apply LMUL algorithm where both exponents are non-zero
    result = np.zeros_like(a_binint)
    valid_mask = (a_exp != 0) & (b_exp != 0)

    if np.any(valid_mask):
        # Only compute for valid inputs
        result[valid_mask] = (
            a_unsigned[valid_mask] + b_unsigned[valid_mask] + lmul_offset
        ) & bitmask

        # Set sign bits
        result[valid_mask] |= sign_result[valid_mask] << (bitwidth - 1)

    return result


def matrix_vector_multiply_lmul(
    weights_binint: np.ndarray, inputs_binint: np.ndarray, dtype: Type[BaseFloat]
) -> np.ndarray:
    """
    Optimized matrix-vector multiplication using vectorized LMUL

    Args:
        weights_binint: Weight matrix in binint representation [output_dim, input_dim]
        inputs_binint: Input vector in binint representation [batch_size, input_dim]
        dtype: Floating-point type

    Returns:
        Result in float representation [batch_size, output_dim]
    """
    batch_size, input_dim = inputs_binint.shape
    output_dim = weights_binint.shape[0]

    # Reshape inputs for broadcasting
    inputs_reshaped = inputs_binint.reshape(batch_size, 1, input_dim)

    # Broadcast weights for batch processing
    weights_broadcast = np.broadcast_to(
        weights_binint.reshape(1, output_dim, input_dim),
        (batch_size, output_dim, input_dim),
    )

    # Apply LMUL to all pairs of weights and inputs
    products_binint = lmul_vectorized(
        weights_broadcast.reshape(-1),
        np.broadcast_to(inputs_reshaped, (batch_size, output_dim, input_dim)).reshape(
            -1
        ),
        dtype,
    ).reshape(batch_size, output_dim, input_dim)

    # Convert products to float for summation
    products_float = binint_to_float_batch(products_binint, dtype)

    # Sum along input dimension
    result_float = np.sum(products_float, axis=2)

    return result_float


def add_bias_vectorized(activations: np.ndarray, bias: np.ndarray) -> np.ndarray:
    """Add bias to activations (in float space)"""
    return activations + bias


def relu_vectorized(x: np.ndarray) -> np.ndarray:
    """Apply ReLU activation"""
    return np.maximum(0, x)


def softmax_vectorized(x: np.ndarray) -> np.ndarray:
    """Apply softmax activation"""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


def optimized_mlp_inference_vectorized(
    inputs_batch: np.ndarray, model_weights: dict, dtype: Type[BaseFloat]
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Highly optimized MLP inference using vectorized operations

    Args:
        inputs_batch: Batch of input vectors [batch_size, input_dim]
        model_weights: Dictionary containing model weights and biases
        dtype: The floating-point data type to use

    Returns:
        Tuple of (predicted_classes, output_probabilities)
    """
    # Pre-convert all weights and biases to binint representation
    fc1_weight_bin = float_to_binint_batch(model_weights["fc1_weight"], dtype)
    fc1_bias = model_weights["fc1_bias"]  # Keep biases as float for addition
    fc2_weight_bin = float_to_binint_batch(model_weights["fc2_weight"], dtype)
    fc2_bias = model_weights["fc2_bias"]  # Keep biases as float for addition

    # Convert inputs to binint
    inputs_bin = float_to_binint_batch(inputs_batch, dtype)

    # First layer: matrix multiplication + bias + ReLU
    h1_float = matrix_vector_multiply_lmul(fc1_weight_bin, inputs_bin, dtype)
    h1_with_bias = add_bias_vectorized(h1_float, fc1_bias)
    a1 = relu_vectorized(h1_with_bias)

    # Convert activations back to binint for next layer
    a1_bin = float_to_binint_batch(a1, dtype)

    # Second layer: matrix multiplication + bias + softmax
    h_out_float = matrix_vector_multiply_lmul(fc2_weight_bin, a1_bin, dtype)
    h_out_with_bias = add_bias_vectorized(h_out_float, fc2_bias)

    # Apply softmax
    output_probs = softmax_vectorized(h_out_with_bias)

    # Get predicted classes
    predicted_classes = np.argmax(output_probs, axis=1)

    return predicted_classes, output_probs


# Example usage
if __name__ == "__main__":
    from hardware_accelerators.dtypes import BF16
    from hardware_accelerators.nn import load_model
    import time

    # Load the model
    model = load_model("../models/mlp_mnist.pth", "cpu")

    # Extract weights and biases
    fc1_weight = model.fc1.weight.data.numpy()
    fc1_bias = model.fc1.bias.data.numpy()
    fc2_weight = model.fc2.weight.data.numpy()
    fc2_bias = model.fc2.bias.data.numpy()

    # Store weights in a dictionary
    model_weights = {
        "fc1_weight": fc1_weight,
        "fc1_bias": fc1_bias,
        "fc2_weight": fc2_weight,
        "fc2_bias": fc2_bias,
    }

    # Get batch of input data
    batch_size = 128
    inputs_batch, labels = get_batch(batch_size)

    # Run standard NumPy inference for comparison
    start_time = time.time()

    # Vectorized standard NumPy implementation
    h1_numpy = inputs_batch @ fc1_weight.T + fc1_bias
    a1_numpy = np.maximum(0, h1_numpy)
    h_out_numpy = a1_numpy @ fc2_weight.T + fc2_bias
    exp_x = np.exp(h_out_numpy - np.max(h_out_numpy, axis=1, keepdims=True))
    a_out_numpy = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    numpy_predictions = np.argmax(a_out_numpy, axis=1)

    numpy_time = time.time() - start_time
    print(
        f"Standard NumPy inference time for batch of {batch_size}: {numpy_time:.4f} seconds"
    )

    # Run optimized inference using vectorized operations
    start_time = time.time()
    predicted_classes, output_probs = optimized_mlp_inference_vectorized(
        inputs_batch, model_weights, BF16
    )
    optimized_time = time.time() - start_time

    print(
        f"Optimized vectorized inference time for batch of {batch_size}: {optimized_time:.4f} seconds"
    )
    print(f"Speedup: {numpy_time / optimized_time:.2f}x")

    # Compare results
    match_count = np.sum(predicted_classes == numpy_predictions)
    print(
        f"Prediction match rate: {match_count}/{batch_size} ({match_count/batch_size*100:.2f}%)"
    )

Standard NumPy inference time for batch of 128: 0.0949 seconds
Optimized vectorized inference time for batch of 128: 1.5121 seconds
Speedup: 0.06x
Prediction match rate: 3/128 (2.34%)


In [35]:
from hardware_accelerators.simulation.matrix_utils import convert_array_dtype

bin_array = convert_array_dtype(np.array([0, 1, 2, 3]), BF16)

[BF16(binint=x).decimal_approx for x in lmul_vectorized(bin_array, bin_array, BF16)]

[0.0, 1.0625, 4.25, 8.5]

In [38]:
import numpy as np
from typing import Type, Tuple, Dict, Any
from hardware_accelerators.dtypes import BaseFloat, BF16, Float32


def convert_array_dtype(values: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """
    Convert array of float values to array of binint representations using the dtype class

    Args:
        values: NumPy array of float values
        dtype: Target floating-point type

    Returns:
        NumPy array of binint representations
    """
    # Flatten the array for processing
    original_shape = values.shape
    flat_values = values.flatten()

    # Create array to hold results
    result = np.zeros(
        flat_values.shape, dtype=np.uint32 if dtype.bitwidth() == 32 else np.uint16
    )

    # Convert each value using the dtype class
    for i, val in enumerate(flat_values):
        result[i] = dtype(val).binint

    # Reshape back to original shape
    return result.reshape(original_shape)


def convert_binint_to_float(binints: np.ndarray, dtype: Type[BaseFloat]) -> np.ndarray:
    """
    Convert array of binint representations back to float values using the dtype class

    Args:
        binints: NumPy array of binint representations
        dtype: Source floating-point type

    Returns:
        NumPy array of float values
    """
    # Flatten the array for processing
    original_shape = binints.shape
    flat_binints = binints.flatten()

    # Create array to hold results
    result = np.zeros(flat_binints.shape, dtype=np.float32)

    # Convert each binint using the dtype class
    for i, binint in enumerate(flat_binints):
        result[i] = float(dtype(binint=int(binint)))

    # Reshape back to original shape
    return result.reshape(original_shape)


def lmul_vectorized(
    a_binint: np.ndarray, b_binint: np.ndarray, dtype: Type[BaseFloat]
) -> np.ndarray:
    """
    Vectorized implementation of LMUL algorithm

    Args:
        a_binint: NumPy array of binint representations
        b_binint: NumPy array of binint representations
        dtype: Floating-point type (BF16 or Float32)

    Returns:
        NumPy array of binint representations of results
    """
    if dtype == BF16:
        # Constants for BF16
        bitmask = 0x7FFF  # 15 bits (excluding sign)
        bitwidth = 16
        mantissa_bits = 7
        lmul_offset = get_combined_offset(8, 7, True)
    elif dtype == Float32:
        # Constants for Float32
        bitmask = 0x7FFFFFFF  # 31 bits (excluding sign)
        bitwidth = 32
        mantissa_bits = 23
        lmul_offset = get_combined_offset(8, 23, True)
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")

    # Create output array
    result = np.zeros_like(a_binint)

    # Flatten arrays for processing
    a_flat = a_binint.flatten()
    b_flat = b_binint.flatten()
    result_flat = result.flatten()

    # Process each pair of values
    for i in range(len(a_flat)):
        a_val = int(a_flat[i])
        b_val = int(b_flat[i])

        # Extract sign bits
        sign_a = (a_val >> (bitwidth - 1)) & 0x1
        sign_b = (b_val >> (bitwidth - 1)) & 0x1
        sign_result = sign_a ^ sign_b

        # Clear sign bits
        a_unsigned = a_val & bitmask
        b_unsigned = b_val & bitmask

        # Check for zero exponents
        a_exp = a_unsigned >> mantissa_bits
        b_exp = b_unsigned >> mantissa_bits

        if a_exp == 0 or b_exp == 0:
            result_flat[i] = 0
        else:
            # Apply LMUL algorithm
            result_val = (a_unsigned + b_unsigned + lmul_offset) & bitmask
            result_val |= sign_result << (bitwidth - 1)
            result_flat[i] = result_val

    return result.reshape(a_binint.shape)


def batch_matrix_vector_multiply(
    weights_binint: np.ndarray, inputs_binint: np.ndarray, dtype: Type[BaseFloat]
) -> np.ndarray:
    """
    Perform matrix-vector multiplication for a batch of inputs using LMUL

    Args:
        weights_binint: Weight matrix in binint representation [output_dim, input_dim]
        inputs_binint: Input vectors in binint representation [batch_size, input_dim]
        dtype: Floating-point type

    Returns:
        Result in float representation [batch_size, output_dim]
    """
    batch_size, input_dim = inputs_binint.shape
    output_dim, input_dim_w = weights_binint.shape

    if input_dim != input_dim_w:
        raise ValueError(
            f"Dimension mismatch: inputs {input_dim} vs weights {input_dim_w}"
        )

    # Initialize result array
    result_float = np.zeros((batch_size, output_dim), dtype=np.float32)

    # Process each batch item and output neuron
    for b in range(batch_size):
        for o in range(output_dim):
            # Multiply weights with inputs using LMUL
            products_binint = lmul_vectorized(
                np.broadcast_to(weights_binint[o], input_dim), inputs_binint[b], dtype
            )

            # Convert products to float for summation
            products_float = convert_binint_to_float(products_binint, dtype)

            # Sum the products
            result_float[b, o] = np.sum(products_float)

    return result_float


def optimized_mlp_inference(
    inputs_batch: np.ndarray, model_weights: dict, dtype: Type[BaseFloat]
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Optimized MLP inference using LMUL for matrix multiplications

    Args:
        inputs_batch: Batch of input vectors [batch_size, input_dim]
        model_weights: Dictionary containing model weights and biases
        dtype: The floating-point data type to use

    Returns:
        Tuple of (predicted_classes, output_probabilities)
    """
    # Pre-convert all weights to binint representation
    fc1_weight_bin = convert_array_dtype(model_weights["fc1_weight"], dtype)
    fc1_bias = model_weights["fc1_bias"]  # Keep biases as float for addition
    fc2_weight_bin = convert_array_dtype(model_weights["fc2_weight"], dtype)
    fc2_bias = model_weights["fc2_bias"]  # Keep biases as float for addition

    # Convert inputs to binint
    inputs_bin = convert_array_dtype(inputs_batch, dtype)

    # First layer: matrix multiplication + bias + ReLU
    h1_float = batch_matrix_vector_multiply(fc1_weight_bin, inputs_bin, dtype)
    h1_with_bias = h1_float + fc1_bias  # Add bias
    a1 = np.maximum(0, h1_with_bias)  # ReLU

    # Convert activations back to binint for next layer
    a1_bin = convert_array_dtype(a1, dtype)

    # Second layer: matrix multiplication + bias
    h_out_float = batch_matrix_vector_multiply(fc2_weight_bin, a1_bin, dtype)
    h_out_with_bias = h_out_float + fc2_bias  # Add bias

    # Apply softmax
    exp_x = np.exp(h_out_with_bias - np.max(h_out_with_bias, axis=1, keepdims=True))
    output_probs = exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Get predicted classes
    predicted_classes = np.argmax(output_probs, axis=1)

    return predicted_classes, output_probs


# Example usage
if __name__ == "__main__":
    from hardware_accelerators.dtypes import BF16
    from hardware_accelerators.nn import load_model
    import time

    # Load the model
    model = load_model("../models/mlp_mnist.pth", "cpu")

    # Extract weights and biases
    fc1_weight = model.fc1.weight.data.numpy()
    fc1_bias = model.fc1.bias.data.numpy()
    fc2_weight = model.fc2.weight.data.numpy()
    fc2_bias = model.fc2.bias.data.numpy()

    # Store weights in a dictionary
    model_weights = {
        "fc1_weight": fc1_weight,
        "fc1_bias": fc1_bias,
        "fc2_weight": fc2_weight,
        "fc2_bias": fc2_bias,
    }

    # Get batch of input data
    batch_size = 8  # Smaller batch for testing
    inputs_batch, _ = get_batch(batch_size)

    # Run standard NumPy inference for comparison
    start_time = time.time()

    ###CHANGE: ENSURE THAT THE DIMENSIONS ARE CORRECT
    print("inputs_batch shape:", inputs_batch.shape, type(inputs_batch))
    print("fc1_weight shape:", fc1_weight.shape)
    print("fc1_bias shape:", fc1_bias.shape)
    
    # Also check if inputs_batch is a proper 2D numeric array
    print("inputs_batch dtype:", inputs_batch.dtype)
    print("Is inputs_batch object dtype?", inputs_batch.dtype == object)
    ###
    
    # Vectorized standard NumPy implementation
    h1_numpy = inputs_batch @ fc1_weight.T + fc1_bias
    a1_numpy = np.maximum(0, h1_numpy)
    h_out_numpy = a1_numpy @ fc2_weight.T + fc2_bias
    exp_x = np.exp(h_out_numpy - np.max(h_out_numpy, axis=1, keepdims=True))
    a_out_numpy = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    numpy_predictions = np.argmax(a_out_numpy, axis=1)

    numpy_time = time.time() - start_time
    print(
        f"Standard NumPy inference time for batch of {batch_size}: {numpy_time:.4f} seconds"
    )

    # Run optimized inference
    start_time = time.time()
    predicted_classes, output_probs = optimized_mlp_inference(
        inputs_batch, model_weights, BF16
    )
    optimized_time = time.time() - start_time

    print(
        f"Optimized inference time for batch of {batch_size}: {optimized_time:.4f} seconds"
    )
    print(f"Speedup: {numpy_time / optimized_time:.2f}x")

    # Compare results
    match_count = np.sum(predicted_classes == numpy_predictions)
    print(
        f"Prediction match rate: {match_count}/{batch_size} ({match_count/batch_size*100:.2f}%)"
    )

    # Print detailed comparison for first few examples
    print("\nDetailed comparison for first 5 examples:")
    for i in range(min(5, batch_size)):
        print(f"Example {i}:")
        print(f"  NumPy prediction: {numpy_predictions[i]}")
        print(f"  Optimized prediction: {predicted_classes[i]}")
        print(f"  Top NumPy probabilities: {np.sort(a_out_numpy[i])[-3:]}")
        print(f"  Top Optimized probabilities: {np.sort(output_probs[i])[-3:]}")

inputs_batch shape: (8, 784) <class 'numpy.ndarray'>
fc1_weight shape: (128, 784)
fc1_bias shape: (128,)
inputs_batch dtype: float32
Is inputs_batch object dtype? False
Standard NumPy inference time for batch of 8: 0.7960 seconds
Optimized inference time for batch of 8: 5.6023 seconds
Speedup: 0.14x
Prediction match rate: 0/8 (0.00%)

Detailed comparison for first 5 examples:
Example 0:
  NumPy prediction: 7
  Optimized prediction: 8
  Top NumPy probabilities: [3.2747920e-07 4.0034397e-06 9.9999571e-01]
  Top Optimized probabilities: [0.10660665 0.11183428 0.12050679]
Example 1:
  NumPy prediction: 2
  Optimized prediction: 8
  Top NumPy probabilities: [4.4528181e-07 1.6522257e-05 9.9998283e-01]
  Top Optimized probabilities: [0.10660665 0.11183428 0.12050679]
Example 2:
  NumPy prediction: 1
  Optimized prediction: 8
  Top NumPy probabilities: [1.5195979e-04 2.6030971e-03 9.9687707e-01]
  Top Optimized probabilities: [0.10660665 0.11183428 0.12050679]
Example 3:
  NumPy prediction: 0


In [42]:
import pyinstrument
profiler = pyinstrument.Profiler(interval=0.000001)

profiler.start()
lmulcls(0, 0)
profiler.stop()
print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 16:35:30  Samples:  326
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.034     CPU time: 0.035
/   _/                      v5.0.1

Profile at /tmp/ipykernel_1177/3627724169.py:4

0.033 ZMQInteractiveShell.run_ast_nodes  IPython/core/interactiveshell.py:3349
├─ 0.025 <module>  /tmp/ipykernel_1177/3627724169.py:1
│  ├─ 0.023 FastLmul.__call__  /tmp/ipykernel_1177/493113130.py:30
│  │  ├─ 0.020 BF16.__init__  hardware_accelerators/dtypes/base.py:30
│  │  │  ├─ 0.020 BF16._init_from_value  hardware_accelerators/dtypes/base.py:102
│  │  │  │  ├─ 0.017 BF16._update_all_representations  hardware_accelerators/dtypes/base.py:144
│  │  │  │  │  ├─ 0.009 BF16._binary_to_decimal  hardware_accelerators/dtypes/bfloat16.py:73
│  │  │  │  │  │  ├─ 0.004 [self]  hardware_accelerators/dtypes/bfloat16.py
│  │  │  │  │  │  ├─ 0.004 <genexpr>  hardware_accelerators/dtypes/bfloat16.py:76
│  │  │  │  │  │  └─ 0.001 BF16._bf16_parts_to_float32  hardware_accele

In [43]:
%%time
0.8 * 0.3

CPU times: user 16 µs, sys: 1e+03 ns, total: 17 µs
Wall time: 38.1 µs


0.24

In [45]:
#run_lmul(0.1, 4, fast=True)
run_lmul(0.1, 4)

Expected: 0.3984375
0.1 + 4 = 0.4140625
0011111011010100


0.4140625

In [47]:
import numpy as np
from typing import List, Type
from hardware_accelerators.dtypes import BaseFloat, BF16


def matrix_vector_multiply_with_lmul(
    A: np.ndarray, v: np.ndarray, dtype: Type[BaseFloat]
) -> np.ndarray:
    """
    Perform matrix-vector multiplication using FastLmul for individual multiplications

    Args:
        A: Matrix as numpy array
        v: Vector as numpy array
        dtype: The floating-point data type to use with FastLmul

    Returns:
        Result vector as numpy array
    """
    # Initialize the FastLmul instance
    lmul = FastLmul(dtype)

    # Get dimensions
    m, n = A.shape

    # Verify that the matrix and vector can be multiplied
    if v.shape[0] != n:
        raise ValueError(
            f"Dimensions don't match for multiplication: A is {m}x{n}, v is {v.shape[0]}"
        )

    # Initialize result vector with zeros
    result = np.zeros(m)

    # Perform matrix-vector multiplication
    for i in range(m):
        sum_val = 0.0
        for j in range(n):
            # Use FastLmul's run method instead of regular multiplication
            product = lmul(A[i, j], v[j])
            # product = lmul.run(A[i, j], v[j])
            sum_val += product
        result[i] = sum_val

    return result


def vector_add(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Simple vector addition without using lmul"""
    return a + b


def relu(x: np.ndarray) -> np.ndarray:
    """ReLU activation function"""
    return np.maximum(0, x)


def softmax(x: np.ndarray) -> np.ndarray:
    """Softmax activation function"""
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / exp_x.sum()


def simulate_mlp_inference_with_lmul(
    input_data: np.ndarray, model_weights: dict, dtype: Type[BaseFloat]
) -> int:
    """
    Simulate MLP inference using FastLmul for matrix multiplications

    Args:
        input_data: Input vector
        model_weights: Dictionary containing model weights and biases
        dtype: The floating-point data type to use with FastLmul

    Returns:
        Predicted class index
    """
    # Extract weights and biases
    fc1_weight = model_weights["fc1_weight"]
    fc1_bias = model_weights["fc1_bias"]
    fc2_weight = model_weights["fc2_weight"]
    fc2_bias = model_weights["fc2_bias"]

    # First layer: matrix multiplication + bias + ReLU
    h1 = matrix_vector_multiply_with_lmul(fc1_weight, input_data, dtype)
    h1 = vector_add(h1, fc1_bias)
    a1 = relu(h1)

    # Second layer: matrix multiplication + bias + softmax
    h_out = matrix_vector_multiply_with_lmul(fc2_weight, a1, dtype)
    h_out = vector_add(h_out, fc2_bias)
    a_out = softmax(h_out)

    # Get the predicted class
    predicted_class = np.argmax(a_out)

    return predicted_class, a_out


# Main execution
if __name__ == "__main__":
    from hardware_accelerators.dtypes import BF16
    from hardware_accelerators.nn import load_model

    # Load the model
    model = load_model("../models/mlp_mnist.pth", "cpu")

    # Extract weights and biases
    fc1_weight = model.fc1.weight.data.numpy()
    fc1_bias = model.fc1.bias.data.numpy()
    fc2_weight = model.fc2.weight.data.numpy()
    fc2_bias = model.fc2.bias.data.numpy()

    # Store weights in a dictionary
    model_weights = {
        "fc1_weight": fc1_weight,
        "fc1_bias": fc1_bias,
        "fc2_weight": fc2_weight,
        "fc2_bias": fc2_bias,
    }

    # Get input data (assuming get_activation() returns a sample from MNIST)
    inputs = get_activation()

    # Run inference using standard numpy operations
    h1_numpy = inputs @ fc1_weight.T + fc1_bias
    a1_numpy = np.maximum(0, h1_numpy)
    h_out_numpy = a1_numpy @ fc2_weight.T + fc2_bias
    a_out_numpy = softmax(h_out_numpy)
    predicted_class_numpy = np.argmax(a_out_numpy)

    print(f"Standard NumPy Inference - Predicted class: {predicted_class_numpy}")

    # Run inference using FastLmul
    predicted_class_lmul, a_out_lmul = simulate_mlp_inference_with_lmul(
        inputs, model_weights, BF16
    )

    print(f"FastLmul Inference (BF16) - Predicted class: {predicted_class_lmul}")

    # Compare the output probabilities
    print("\nOutput Probabilities Comparison:")
    for i in range(len(a_out_numpy)):
        print(
            f"Class {i}: NumPy = {a_out_numpy[i]:.6f}, FastLmul = {a_out_lmul[i]:.6f}, "
            f"Diff = {abs(a_out_numpy[i] - a_out_lmul[i]):.6f}"
        )

    # Check if the predictions match
    if predicted_class_numpy == predicted_class_lmul:
        print("\nBoth methods predicted the same class! ✓")
    else:
        print(
            f"\nPrediction mismatch: NumPy predicted {predicted_class_numpy}, "
            f"FastLmul predicted {predicted_class_lmul} ✗"
        )

Standard NumPy Inference - Predicted class: 7
FastLmul Inference (BF16) - Predicted class: 7

Output Probabilities Comparison:
Class 0: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 1: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 2: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 3: NumPy = 0.000004, FastLmul = 0.000005, Diff = 0.000001
Class 4: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 5: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 6: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 7: NumPy = 0.999996, FastLmul = 0.999994, Diff = 0.000001
Class 8: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000
Class 9: NumPy = 0.000000, FastLmul = 0.000000, Diff = 0.000000

Both methods predicted the same class! ✓


In [48]:
systolic_sim = SystolicArraySimulator(8, multiplier=lmul_fast)

In [50]:
model = load_model("../models/mlp_mnist_bf16.pth")
weights = model.fc1.weight.data.numpy(force=True)
weights

array([[-0.00762939, -0.02355957,  0.04174805, ...,  0.01428223,
        -0.02355957,  0.0039978 ],
       [ 0.03027344,  0.00610352,  0.04248047, ..., -0.00167847,
         0.03173828,  0.03198242],
       [-0.00056458,  0.01794434,  0.00424194, ...,  0.0279541 ,
         0.01501465,  0.00738525],
       ...,
       [ 0.00288391, -0.02893066,  0.01831055, ..., -0.02929688,
         0.01165771,  0.01397705],
       [ 0.02734375, -0.01879883,  0.02734375, ...,  0.0246582 ,
         0.02062988,  0.02722168],
       [ 0.00787354, -0.00135803,  0.0201416 , ...,  0.02124023,
        -0.03344727, -0.00189972]], shape=(128, 784), dtype=float32)

In [51]:
np.random.seed(42)
fake_acts = np.random.rand(8, 8)
fake_acts

array([[0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
        0.15599452, 0.05808361, 0.86617615],
       [0.60111501, 0.70807258, 0.02058449, 0.96990985, 0.83244264,
        0.21233911, 0.18182497, 0.18340451],
       [0.30424224, 0.52475643, 0.43194502, 0.29122914, 0.61185289,
        0.13949386, 0.29214465, 0.36636184],
       [0.45606998, 0.78517596, 0.19967378, 0.51423444, 0.59241457,
        0.04645041, 0.60754485, 0.17052412],
       [0.06505159, 0.94888554, 0.96563203, 0.80839735, 0.30461377,
        0.09767211, 0.68423303, 0.44015249],
       [0.12203823, 0.49517691, 0.03438852, 0.9093204 , 0.25877998,
        0.66252228, 0.31171108, 0.52006802],
       [0.54671028, 0.18485446, 0.96958463, 0.77513282, 0.93949894,
        0.89482735, 0.59789998, 0.92187424],
       [0.0884925 , 0.19598286, 0.04522729, 0.32533033, 0.38867729,
        0.27134903, 0.82873751, 0.35675333]])

In [52]:
fake_acts2 = la1.copy().reshape(8, -1)[:8, :8]

gt = weights[:8, :8] @ fake_acts2
sim_result = SystolicArraySimulator.matrix_multiply(
    weights[:8, :8], fake_acts, multiplier=lmul_fast
)

NameError: name 'la1' is not defined

In [220]:
print(np.array2string(gt, precision=3, suppress_small=True), "\n")
print(np.array2string(sim_result, precision=3, suppress_small=True), "\n")
print(np.isclose(gt, sim_result, atol=1e-2))

[[ 0.034  0.21   0.02   0.387 -0.132  0.245 -0.062 -0.159]
 [ 0.063  0.223  0.325  0.617  0.272  1.126  0.309  0.252]
 [-0.011 -0.018 -0.057 -0.392 -0.106 -0.305 -0.002  0.036]
 [ 0.009 -0.101 -0.209 -0.316 -0.246 -0.767 -0.445 -0.234]
 [ 0.009  0.081 -0.042  0.041 -0.183 -0.232 -0.001  0.443]
 [-0.056  0.039 -0.207 -0.223 -0.31  -0.809  0.019 -0.014]
 [ 0.008  0.105  0.265  0.17   0.418  0.516  0.368  0.396]
 [-0.062 -0.003 -0.212 -0.596 -0.282 -0.794 -0.058 -0.248]] 

[[ 0.013 -0.004  0.013 -0.014  0.03   0.009  0.027  0.013]
 [ 0.061  0.138  0.093  0.129  0.1    0.056  0.102  0.102]
 [-0.008 -0.007 -0.015 -0.014 -0.014 -0.019 -0.023 -0.02 ]
 [-0.068 -0.16  -0.068 -0.143 -0.09  -0.03  -0.06  -0.078]
 [-0.006  0.019  0.071  0.013  0.008  0.009  0.032  0.028]
 [-0.016 -0.048  0.004 -0.05  -0.029 -0.019 -0.047 -0.02 ]
 [ 0.034  0.106  0.096  0.118  0.055  0.06   0.054  0.098]
 [-0.02  -0.058 -0.056 -0.074 -0.046 -0.049 -0.081 -0.059]] 

[[False False  True False False False False False]

# Emulating lmul with pytorch


In [39]:
import torch
import torch.nn as nn

In [176]:
A, B = 3.182, 0

# dtype = torch.bfloat16
dtype = torch.float32


def lmul(a, b, dtype):
    torch_viewmap = {
        torch.float32: torch.uint32,
        torch.bfloat16: torch.uint16,
        torch.float16: torch.uint16,
    }

    lmul_offset = {
        torch.bfloat16: 16248,
        torch.float32: 1064828928,
    }

    a = torch.tensor(a, dtype=dtype).view(torch_viewmap[dtype]).item()
    b = torch.tensor(b, dtype=dtype).view(torch_viewmap[dtype]).item()

    lmul_ab = a + b - lmul_offset[dtype]
    # print(format(lmul_ab, f'0{torch_viewmap[dtype].itemsize * 8}b'))
    lmul_ab = torch.tensor(lmul_ab, dtype=torch_viewmap[dtype]).view(dtype).item()

    return lmul_ab


lmul(A, B, dtype), A * B

(1.9436798631220628e-38, 0.0)

In [69]:
get_combined_offset(8, 23, twos_comp=False)

1064828928

In [44]:
torch.scalar_tensor(A).view(torch_viewmap[dtype])

RuntimeError: self.dim() cannot be 0 to view Float as UInt16 (different element sizes)

In [60]:
import torch


def lmul_matmul(A: torch.Tensor, B: torch.Tensor, dtype=torch.float32):
    if dtype == torch.float32:
        # Use uint32 view, but cast to int64 for arithmetic
        A_int = A.view(torch.uint32).to(torch.int64)
        B_int = B.view(torch.uint32).to(torch.int64)
        offset = 1064828928  # special offset for float32
    elif dtype == torch.bfloat16:
        A_int = A.view(torch.uint16).to(torch.int64)
        B_int = B.view(torch.uint16).to(torch.int64)
        offset = 16248  # special offset for bfloat16
    else:
        raise ValueError("Unsupported dtype")

    # Suppose A is (m, n) and B is (n, p)
    # Expand dimensions so we can broadcast:
    # A_int -> (m, n, 1) and B_int -> (1, n, p)
    prod_int = A_int.unsqueeze(2) + B_int.unsqueeze(0) - offset  # shape: (m, n, p)

    # Convert the integer results back to floating point:
    if dtype == torch.float32:
        prod = prod_int.to(torch.uint32).view(torch.float32)
    else:  # bfloat16 case
        prod = prod_int.to(torch.uint16).view(torch.bfloat16)

    # Sum over the reduction dimension to complete the dot product
    return prod.sum(dim=1)


# Example usage:
m, n, p = 2, 3, 4  # for instance
A = torch.randn(m, n, dtype=torch.float32)
B = torch.randn(n, p, dtype=torch.float32)
C = lmul_matmul(A, B, dtype=torch.float32)
print("Approximate product:\n", C)

# Verify with exact product
exact_product = torch.matmul(A, B)
print("Exact product:\n", exact_product)

Approximate product:
 tensor([[ 0.8586,  0.4696, -1.8520,  0.0495],
        [ 1.7733, -0.9895,  0.7423, -4.1074]])
Exact product:
 tensor([[ 0.9098,  0.4175, -1.8140,  0.0773],
        [ 1.7574, -1.0393,  0.7893, -4.1009]])


## Loading mnist data


In [61]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
# Download MNIST test data
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels.numpy()


def get_activation():
    image, _ = next(iter(test_loader))
    image = image.detach().numpy().reshape(-1)
    return image

## Loading model weights


In [62]:
model = load_model("../models/mlp_mnist.pth", "cpu")

fc1_weight = model.fc1.weight.data.numpy()
fc1_bias = model.fc1.bias.data.numpy()
fc2_weight = model.fc2.weight.data.numpy()
fc2_bias = model.fc2.bias.data.numpy()

In [63]:
model = load_model("../models/mlp_mnist.pth", "cpu")

fc1_weight = model.fc1.weight.data.numpy()
fc1_bias = model.fc1.bias.data.numpy()
fc2_weight = model.fc2.weight.data.numpy()
fc2_bias = model.fc2.bias.data.numpy()

inputs = get_activation()

h1 = inputs @ fc1_weight.T + fc1_bias
a1 = np.maximum(0, h1)
h_out = a1 @ fc2_weight.T + fc2_bias
a_out = softmax(h_out)

# get the index of the maximum value
predicted_class = np.argmax(a_out)
print(f"Predicted class: {predicted_class}")

Predicted class: 7


## Numpy inference


In [64]:
inputs = get_activation()

h1 = inputs @ fc1_weight.T + fc1_bias
a1 = np.maximum(0, h1)
h_out = a1 @ fc2_weight.T + fc2_bias
a_out = softmax(h_out)

# get the index of the maximum value
predicted_class = np.argmax(a_out)
print(f"Predicted class: {predicted_class}")

Predicted class: 7


## Lmul inference


In [65]:
inputs, labels = get_batch(1)

lh1 = (
    lmul_matmul(
        torch.tensor(inputs, dtype=torch.float32),
        torch.tensor(fc1_weight.T, dtype=torch.float32),
        torch.float32,
    ).numpy()
    + fc1_bias
)
la1 = np.maximum(0, lh1)
lh_out = (
    lmul_matmul(
        torch.tensor(la1, dtype=torch.float32),
        torch.tensor(fc2_weight.T, dtype=torch.float32),
        torch.float32,
    ).numpy()
    + fc2_bias
)
# la_out = softmax(lh_out)

# # get the index of the maximum value
# predicted_class = np.argmax(la_out)
# print(f"Predicted class: {predicted_class}")

lh_out

array([[inf, nan, inf, inf, nan, nan, inf, inf, nan, inf]], dtype=float32)

In [66]:
lmul_matmul(
    torch.tensor(la1, dtype=torch.float32),
    torch.tensor(fc2_weight.T, dtype=torch.float32),
    torch.float32,
)

tensor([[inf, nan, inf, inf, nan, nan, inf, inf, nan, inf]])

In [67]:
la1

array([[ 0.        ,  0.        ,  3.1820114 ,  0.        ,  0.        ,
         6.688619  ,  6.5706654 ,  0.        ,  0.        ,  2.4044447 ,
         3.5749934 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  3.9750876 ,  0.        ,  0.        ,
         0.        ,  4.382746  ,  7.232212  ,  0.        ,  0.        ,
         0.        ,  2.6718044 ,  0.        ,  4.2498627 ,  0.30033502,
         1.1679474 ,  0.        ,  1.2926131 ,  0.        ,  0.        ,
         5.2734685 ,  0.        ,  0.        ,  0.        ,  0.        ,
         9.827601  ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  9.037512  ,  0.        ,  7.0829687 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.66314924,  0.        ,  0.        ,  0.        ,  0.        ,
         0.31634068,  0.        ,  0.        ,  0. 

In [68]:
h20 = 0
lh20 = 0
for a, w in zip(la1[0], fc2_weight[0]):
    product = a * w
    lproduct = run_lmul(a, w, Float32)
    print(f"{a:.3f}, {w:.3f}")
    print(f"{product:.3f}, {lproduct:.3f}\n")
    h20 += product
    lh20 += lproduct
h20, lh20

Expected: 0.0
0.0 + -0.06308482587337494 = 0.0
00000000000000000000000000000000
0.000, -0.063
-0.000, 0.000

Expected: 0.0
0.0 + 0.06475932896137238 = 0.0
00000000000000000000000000000000
0.000, 0.065
0.000, 0.000

Expected: 0.19324742257595062
3.182011365890503 + 0.06073121726512909 = 0.1996130794286728
00111110010011000110011101011111
3.182, 0.061
0.193, 0.200

Expected: 0.0
0.0 + 0.10466907918453217 = 0.0
00000000000000000000000000000000
0.000, 0.105
0.000, 0.000

Expected: 0.0
0.0 + 0.03408030420541763 = 0.0
00000000000000000000000000000000
0.000, 0.034
0.000, 0.000

Expected: -0.1398218870162964
6.688619136810303 + -0.02090444788336754 = -0.13406743109226227
10111110000010010100100011111001
6.689, -0.021
-0.140, -0.134

Expected: 0.26932308077812195
6.57066535949707 + 0.04098870977759361 = 0.2542012631893158
00111110100000100010011010101011
6.571, 0.041
0.269, 0.254

Expected: 0.0
0.0 + -0.03825875744223595 = 0.0
00000000000000000000000000000000
0.000, -0.038
-0.000, 0.000

Expect

(np.float32(-5.1689677), -5.280333912931383)

In [69]:
lmul(0, 0.47, torch.float32)

AttributeError: 'torch.dtype' object has no attribute 'exponent_bits'

In [70]:
fc2_bias

array([-0.01075647, -0.14156555,  0.01885387, -0.1005828 ,  0.08457243,
       -0.13470922, -0.09962139, -0.13597849,  0.15926038,  0.0367002 ],
      dtype=float32)

In [71]:
h_out

array([ -5.2412014, -12.86712  ,  -1.4991012,   1.0043828, -17.54349  ,
        -8.378266 , -18.727932 ,  13.432735 ,  -7.506784 ,  -3.5312648],
      dtype=float32)

In [72]:
final_output = []
for col in range(fc2_weight.shape[0]):
    output = 0
    for row in range(fc2_weight.shape[1]):
        output += lmul(la1[0, row], fc2_weight.T[row, col], torch.float32)
    final_output.append(output)
final_output

AttributeError: 'torch.dtype' object has no attribute 'exponent_bits'